In [1]:
from Raspi_MotorHAT import Raspi_PWM_Servo_Driver
import time

# это PID регуляторы. Нужны для того чтобы камера не дёргалась 
from simple_pid import PID
pid_ver = PID(0.3, 0.02, 0.01, setpoint=0, output_limits=(-200, 200))
pid_hor = PID(0.3, 0.02, 0.01, setpoint=0, output_limits=(-200, 200))

cam_up=800
cam_down=2000
cam_left=2650
cam_right=490
steer_left=2700
steer_right=2150

cam_ver = (cam_up + cam_down)//2
cam_hor = (cam_left + cam_right)//2

pwm1 = Raspi_PWM_Servo_Driver.PWM(0x70)
pwm1.setPWMFreq(250)
pwm1.setPWM(14, 0, 0)
def update_cam(err_hor, err_ver, dt=None):
    global cam_ver, cam_hor, pid_ver, pid_hor
    cam_ver += int(pid_ver(-err_ver, dt))
    cam_hor += int(pid_hor(-err_hor, dt))
    if cam_ver < cam_up:
        cam_ver = cam_up
        pid_ver.reset()
    if cam_ver > cam_down:
        cam_ver = cam_down
        pid_ver.reset()
    if cam_hor < cam_right:
        cam_hor = cam_right
        pid_hor.reset()
    if cam_hor > cam_left:
        cam_hor = cam_left
        pid_hor.reset()
    pwm1.setPWM(0, 0, cam_ver)
    pwm1.setPWM(1, 0, cam_hor)
update_cam(0, 0)

In [2]:
import time
from io import BytesIO
from time import sleep
from picamera import PiCamera
from PIL import Image
from IPython.display import clear_output
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.colors import hsv_to_rgb
import imutils


try: camera
except NameError:
    camera = PiCamera()
    camera.resolution = (320, 240)
    camera.framerate = 24
    camera.start_preview()
    time.sleep(2)


image = np.empty((240 * 320 * 3,), dtype=np.uint8)
image = image.reshape((240, 320, 3))
prev_run = time.time()
for _ in camera.capture_continuous(image, format='rgb', use_video_port=True):
    clear_output(wait=True)

    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_RGB2HSV)
    
    # lower mask (0-10)
    lower_red = np.array([0,110,110])
    upper_red = np.array([15,255,255])
    mask0 = cv2.inRange(hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([165,110,110])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # join my masks
    mask = mask0+mask1
    
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    img = image // 4
    
    # ищем контуры
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    # only proceed if at least one contour was found
    x, y = None, None
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        # only proceed if the radius meets a minimum size
        if radius > 5:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(img, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
        
    if x is None or y is None:
        print("No red")
        pid_ver.reset()
        pid_ver.reset()
        continue
    
    cam_x_angle, cam_y_angle = 170, 127
    angle_x_err = (x/320 - 0.5)*cam_x_angle
    angle_y_err = (y/240 - 0.5)*cam_y_angle
    
    x_err = -(cam_left - cam_right)*angle_x_err//180
    y_err = (cam_down - cam_up)*angle_y_err//100
    update_cam(x_err, y_err)
    
    print(f"dT: {(time.time() - prev_run):.5f}, angle error: {(angle_x_err):+.5f}x {angle_y_err:+.5f}y")
    prev_run = time.time()  


KeyboardInterrupt: 